In [98]:
import os
import sqlite3
import chromadb
import ollama
from app.helpers import parse_abs_data
from app.helpers import extract_citations
from app.helpers import insert_new_paper
from app.helpers import remove_paper
from app.helpers import reconstruct_paper_id
from app.helpers import add_document
from app.helpers import remove_document
import logging

logging.basicConfig(level=logging.ERROR)


In [99]:
file_path = '/Users/saianoopavunuri/Documents/cs6400/project/cit-HepTh-abstracts/2001/0101005.abs'
paper_id = '101005'
file_name = '0101005.abs'
with open(file_path, 'r', encoding='utf-8') as file:
    abs_data = file.read()
    parsed_data = parse_abs_data(abs_data)
    # Remove leading zeros from filename
    parsed_data['paper_id'] = str(int(file_name.replace('.abs', '')))
citations_from, citations_to = extract_citations(paper_id, '/Users/saianoopavunuri/Documents/cs6400/project/cit-HepTh.txt')

In [100]:
parsed_data

{'paper_id': '101005',
 'authors': 'Khaled Saaidi, Mohammad Khorrami',
 'title': 'Nonlocal two-dimensional Yang-Mills- and generalized Yang-Mills-theories',
 'comments': '13 pages, LaTeX 2.09',
 'subj_class': None,
 'journal_ref': 'Int.J.Mod.Phys. A15 (2000) 4749-4759',
 'abstract': 'A generalization of the two-dimensional Yang-Mills and generalized Yang-Mills\ntheory is introduced in which the building B-F theory is nonlocal in the\nauxiliary field. The classical and quantum properties of this nonlocal\ngeneralization are investigated and it is shown that for large gauge groups,\nthere exist a simple correspondence between the properties a nonlocal theory\nand its corresponding local theory.'}

In [101]:
# Connect to the existing database
connection = sqlite3.connect("papers.db")

In [102]:
# Create a cursor object
cursor = connection.cursor()
query = f"select * from papers where paper_id = {paper_id};"
# query = f"select * from citations where from_paper_id = {paper_id};"
# query = f"select * from co_citations where paper_1_id = {paper_id};"
results = cursor.execute(query)
for row in results:
    print(row)


('101005', 'Nonlocal two-dimensional Yang-Mills- and generalized Yang-Mills-theories', 'Khaled Saaidi, Mohammad Khorrami', '13 pages, LaTeX 2.09', None, 'Int.J.Mod.Phys. A15 (2000) 4749-4759', 'A generalization of the two-dimensional Yang-Mills and generalized Yang-Mills\ntheory is introduced in which the building B-F theory is nonlocal in the\nauxiliary field. The classical and quantum properties of this nonlocal\ngeneralization are investigated and it is shown that for large gauge groups,\nthere exist a simple correspondence between the properties a nonlocal theory\nand its corresponding local theory.', 3, '2000-12-30')


In [103]:
remove_paper(connection, paper_id)

Document with paper_id 0101005 removed successfully.
Document with paper_id 0101005 removed successfully.


In [108]:
client = chromadb.PersistentClient(path="full_data/")
collection_docs = client.get_or_create_collection(name="docs")
collection_docs.get('0101005')

{'ids': ['0101005'],
 'embeddings': None,
 'documents': ['A generalization of the two-dimensional Yang-Mills and generalized Yang-Mills\ntheory is introduced in which the building B-F theory is nonlocal in the\nauxiliary field. The classical and quantum properties of this nonlocal\ngeneralization are investigated and it is shown that for large gauge groups,\nthere exist a simple correspondence between the properties a nonlocal theory\nand its corresponding local theory.'],
 'uris': None,
 'data': None,
 'metadatas': [None],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [105]:
# Create a cursor object
cursor = connection.cursor()
# query = f"select * from papers where paper_id = {paper_id};"
# query = f"select * from citations where to_paper_id = {paper_id};"
query = f"select * from co_citations where paper_1_id = {paper_id};"

results = cursor.execute(query)
for row in results:
    print(row)


In [107]:
# :param conn: SQLite database connection.
# :param paper_metadata: Dictionary containing paper metadata (e.g., paper_id, title, authors, etc.).
# :param citations_from: List of papers that this paper cites.
# :param citations_to: List of papers that cite this paper.
# :param publication_date: Date when the paper was published.

insert_new_paper(connection, parsed_data, citations_from, citations_to, '2000-12-30')

Document with paper_id 0101005 added successfully.
Document with paper_id 0101005 added successfully.


In [63]:
connection.commit()
cursor.close()
connection.close()

In [10]:
# Initialize the client and collections
client = chromadb.PersistentClient(path="full_data/")
collection_docs = client.get_or_create_collection(name="docs")
collection_authors = client.get_or_create_collection(name="authors")


In [11]:
paper_id = '101005'
paper_id = reconstruct_paper_id(paper_id)
print(paper_id)

0101005


In [13]:
# Example usage
# Assuming ollama.embeddings is a function to fetch embeddings
response = ollama.embeddings(model="mxbai-embed-large", prompt=parsed_data['abstract'])
embedding = response["embedding"]

# Remove document if it exists
remove_document(collection_docs, paper_id)

# Add a new document
add_document(collection_docs, paper_id, parsed_data['abstract'], embedding)


response = ollama.embeddings(model="mxbai-embed-large", prompt=parsed_data['authors'])
embedding = response["embedding"]

# Remove document if it exists
remove_document(collection_authors, paper_id)

# Add a new document
add_document(collection_authors, paper_id, parsed_data['authors'], embedding)

Add of existing embedding ID: 0101005
Add of existing embedding ID: 0101005


Document with paper_id 0101005 removed successfully.
Document with paper_id 0101005 added successfully.
Document with paper_id 0101005 removed successfully.
Document with paper_id 0101005 added successfully.
